## FII_SMART Multi-evidence Natural Language Inference for Clinical Trial Data


In [155]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [156]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: nvidia-smi: command not found


In [157]:
import sys
import os

DEBUG1=False
DEBUG2=False

PREFIXG="/content/"
PREFIX="/content/"

TRAIN_DATA=PREFIXG+"train.csv"
TEST_DATA=PREFIXG+"dev.csv"
TEST_DATA2=PREFIXG+"test.csv"

Clean files for new run

In [158]:
from google.colab import files

!ls
!rm test.*
!rm train.*
!rm dev.*
!rm subm*.*
!rm test-*.*
!rm results-*.*
!rm -rf CT_data
!rm CT_data.zip
!rm dataset.zip

CT_data      NLI4CT_numerical_instances_dataset.json	    test.csv
CT_data.zip  pairs_NLI4CT_numerical_instances_dataset.json  test.json
dataset.zip  results_1_20230603185425.json		    train.csv
dev.csv      sample_data				    train.json
dev.json     test
rm: cannot remove 'subm*.*': No such file or directory
rm: cannot remove 'test-*.*': No such file or directory
rm: cannot remove 'results-*.*': No such file or directory


In [159]:
!pip install transformers > /dev/null 2>&1
!wget https://github.com/volosincu/FII_Smart__Semeval2023/raw/main/dataset.zip
!unzip /content/dataset.zip
!unzip /content/CT_data.zip  > /dev/null 2>&1

--2023-06-03 20:43:18--  https://github.com/volosincu/FII_Smart__Semeval2023/raw/main/dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/volosincu/FII_Smart__Semeval2023/main/dataset.zip [following]
--2023-06-03 20:43:19--  https://raw.githubusercontent.com/volosincu/FII_Smart__Semeval2023/main/dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2479891 (2.4M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]   2.36M  --.-KB/s    in 0.06s   

2023-06-03 20:43:19 (37.0 MB/s) - ‘dataset.zip’ saved [2479891/2479891]

Archive:  /content/datas

In [160]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('english'))

STOP_WORDS_RO = [ ]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Tests for numerical regex  

In [161]:
with open('test', encoding='utf-8', mode='w+') as f:
    f.write('\u4500 blah blah blah\n')
    f.seek(0)
    print(repr(f.readline()[:1]))

import unicodedata    
raw_text = "Patients with Platelet count over 100,000/mm\u00ac\u00a8\u201a\u00e2\u2022, ANC "
convert_text = unicodedata.normalize('NFKD', raw_text)
convert_text

'䔀'


'Patients with Platelet count over 100,000/mm¬ ̈‚â•, ANC '

In [162]:
import re

def uninorm(w):
  return unicodedata.normalize('NFKD', w)

def remove_partitions(numInstances, skip=False):
  if skip:
    return numInstances
  b1 = []
  for w in numInstances:
    a0 = numInstances[0:]
    a0.remove(w)
    contained = False
    for x in a0:
      if w.strip() in x.strip():
        contained = True
    if not contained:
      b1.append(w.strip())
  return b1

def containsNumericalInstances(sentence, skip=False):
  # '2,0 mg/dl', '9,7 mg'
  s00 = re.findall(r'[0-9]+,[0-9]+\s{1}[a-zA-Z\/\^\-\:\+\*\=,0-9]+', sentence, re.UNICODE) # ['9,7 mg']  # ['9,7 test']
  
  # s01 = re.findall(r'^[0-9]+\s{0,1}[a-zA-Z0-9]+[\^\/\-_]{1}[a-zA-Z0-9]+|(?:\s)[0-9]+\s{0,1}[a-zA-Z0-9]+[\^\/\-_]{1}[a-zA-Z0-9]+', sentence, re.UNICODE)  # 50 mg/min
  s010 = re.findall(r'^[0-9]+\s{0,1}[a-zA-Z0-9]+[\^\/\-_]{1}[a-zA-Z0-9]+|(?:\s)[0-9]+\s{0,1}[a-zA-Z0-9]+[\^\/\-_]{1}[a-zA-Z0-9]+(?:\s{1}[a-zA-Z]+)', sentence, re.UNICODE)  # 50 mg/min

  s02 = re.findall(r'[^a-zA-Z][0-9]+[\.\,]{1}[0-9%]+\s{0,1}[a-zA-Z]+[\^\/\-_]{1}[a-zA-Z]+', sentence, re.UNICODE)  # 2.0 mg/dl, ' 9.0 g/dL'
  s020 = re.findall(r'[^a-zA-Z\S][0-9]+\s{0,1}[a-zA-Z]+[\^\/\-_]{1}[a-zA-Z\S]+', sentence, re.UNICODE)  # 2.0 mg/dl, ' 300 mg/m2'
  # the ALT 2.5 times , 'Hemoglobin 9.0 g/dL'
  s021 = re.findall(r'\w+\s[0-9]+[\.\,]{1}[0-9%]+\s{0,1}[a-zA-Z]+[\^\/\-_\s]{1}[a-zA-Z0-9]+', sentence, re.UNICODE)

  # '1-21. Inclusion', ['9,7 test'], '9,7 test', '88888888/55 9/8' , '52-75 years', '9/8 mg', '1/50 mg', '2/neu positive', '0-1 ANC', '1,500/mm³ Platelet', '100,000/mm³ Hemoglobin'
  s03 = re.findall(r'[\(\[\{\<]*[0-9]+[,\-\:\+\=\@\#\/][0-9%\.\S]+[\)\]\}\>]{0,1}\s{1}[a-zA-Z\/\^\-\:\+\*\=,0-9]+', sentence, re.UNICODE) 
  # '(300%) than'
  s030 = re.findall(r'[\(\[\{\<]{1}\w*[0-9]+\w*[,\-\:\+\=\@\#\/]*[0-9%]*[\)\]\}\>]{1}\s{0,1}(?:\s{1}[a-zA-Z]+)', sentence, re.UNICODE)
  
  # '8,88888888/55 mg', '9/8 mg', '1/50 mg', '5.5/5.0 11', '3.74/9.45666 11'
  s1 = re.findall(r'[0-9]+[\.\,\-\:\+\=\@\#\/]{0,1}[0-9]*[\/][0-9]+[\.\,\-\:\+\=\@\#\/]{0,1}[0-9]*\s{1}[a-zA-Z\S]+[\^\/\-_]{0,1}[a-zA-Z0-9\S]+', sentence, re.UNICODE) 
  
  # 'y3.222 mg', 'f3.566 mg/kg', 'Ki67 staining'
  s10 = re.findall(r'[a-zA-Z]+[0-9]+[\.\,\-\:\+\=\@\#\/]{0,1}[0-9%]+\s{1}[a-zA-Z]+[\^\/\-_]{0,1}[a-zA-Z]+', sentence, re.UNICODE) 
  
  # '61.1% mg', '65.5 % mg'
  s2 = re.findall(r'[0-9]+[\.\,\-\:\+\=\@\#\/]{0,1}[0-9]*\s{0,1}%\s{1}[a-zA-Z]+[\^\/\-_]{0,1}[a-zA-Z]+', sentence, re.UNICODE) 
  # s21 = re.findall(r'([0-9]+[\.]{0,1}[0-9]*)[\s]{0,1}%[\s]{0,1}[a-zA-z]+', sentence)  # '5.5/5.0', '3.7/9.0', '3.74/9.45666',

  # '5.5/5.0 11 mg ', '3.7/9.0 11 mg ', '3.74/9.45666 11 mg '
  s3 = re.findall(r'[0-9%]+[\.\,\-\:\+\=\@\#\/]{0,1}[0-9%]*[\/][0-9%]+[\.\,\-\:\+\=\@\#\/]{0,1}[0-9%]+(?!\s{1}[a-zA-Z0-9])', sentence, re.UNICODE) 
  
  #'5.5/5.0 11 mg ', '3.7/9.0 11 mg ', '3.74/9.45666 11 mg ',  '8,88888888/55 9/8 ', '3.7/9.0 11  '
  s41 = re.findall(r'[0-9]+[\.\,\-\:\+\=\@\#\/]{0,1}[0-9]*[\/][0-9]+[\.\,\-\:\+\=\@\#\/]{0,1}[0-9]*\s{1}[\s\d\^\/\-_]+[a-zA-Z]{,4}\s{1}', sentence, re.UNICODE)
  #s3 = re.findall(r'[0-9]+[\.]{0,1}[0-9]*\s{0,1}%', sentence) # '61.1%', '65.5 %'

  # ' 2 months of', ' 12 weeks ECOG', ' 200 mg test'
  s50 = re.findall(r'\s[0-9]+\s{1}[a-zA-Z]+(?:\s[a-zA-Z0-9;]+)', sentence, re.UNICODE)
  s51 = re.findall(r'\s{0,1}[0-9]+\s{1}[a-zA-Z]+(?:\s[0-9a-zA-Z;\.\,\:\?\[\(\{\}\]\)\=]+\s)', sentence, re.UNICODE)


  s60 = re.findall(r'[0-9]+[\.\,\-\:\+\=\@\#\/]{1}[0-9]+[\.\,\-\:\+\=\@\#\/]{1}[a-zA-Z][\S]*', sentence, re.UNICODE) #'100,000/mm³'
  s61 = re.findall(r'[0-9]+[\.\,\-\:\+\=\@\#\/]{1}[0-9]+[\.\,\-\:\+\=\@\#\/]{1}[a-zA-Z][\S]*(?:\s[a-zA-Z]+)', sentence, re.UNICODE) #'100,000/mm³'

  # s70 = re.findall(r'[0-9]+[\.,]{1}[0-9]+[\^\/\-_]{1}[a-z][\S]*', sentence) #'100,000/mm³'

  s80 = re.findall(r'[0-9]+[\.\,\-\:\+\=\@\#\/]{1}[0-9]+[\.\,\-\:\+\=\@\#\/]{1}[a-zA-Z][\S]*(?:\s[a-zA-Z]+)', sentence, re.UNICODE) #'100,000/mm³'

  s90 = re.findall(r'[0-9]+[\.\,\-\:\+\=\@\#\/]{1}[0-9]+[\.\,\-\:\+\=\@\#\/]{1}[a-zA-Z][\S]*', sentence, re.UNICODE) #'100,000/mm³'

  # return s51

  strs = s010 + s02 + s020 + s021 + s03 + s030 + s00 + s10 + s1 + s2 + s41 + s50 + s51 + s60 + s61 + s90
  insts = set([s.strip() for s in strs])
  return remove_partitions(list(insts), skip)


a99 = containsNumericalInstances(' Patients should have T1N1-3M0 or T2-4 N0-3M0. Patients with Platelet count over 100,000/mm\u00ac\u00a8\u201a\u00e2\u2022, ANC smaller than 1,700/mm\u00ac\u00a8\u201a\u00e2\u2022 and Hemoglobin between 4 to 5 grams per deciliter to 28 days; are eligible for the primary trial ', True)
print(a99)
print(str(a99)==str(['2-4 N0-3M0', '1-3M0 or', '100,000/mm¬¨‚â•, ANC', '1,700/mm¬¨‚â•', '100,000/mm¬¨‚â•,', '5 grams per', '1,700/mm¬¨‚â• and']))

a98 = containsNumericalInstances("Patients with HER-2/neu positive tumors asymptomatic for 2 months of progression status not  expectancy 12 weeks ECOG performance status 0-1 ANC 1,500/mm³ Platelet count 100,000/mm³ Hemoglobin 9.0 g/dL ", True)
print(a98)
print(str(a98)==str(['100,000/mm³ Hemoglobin', '0-1 ANC', '100,000/mm³', '2/neu positive', '12 weeks ECOG', '9.0 g/dL', '2 months of', '1,500/mm³ Platelet', '1,500/mm³']))


a0 = containsNumericalInstances(" Ki67 staining (cumulative dose) and the ALT 2.5 times daily on days 1-21. Inclusion Criteria: Age 52-75 years old; IdentificationPatients also receive cixutumumab IV over 1-1\u00bd hours on days 1, 8, and 15. Courses repeat ", True)
print(a0)
print(str(a0)==str(['1-21. Inclusion', '52-75 years', '1-1½ hours', 'Ki67 staining']))


a1 = containsNumericalInstances("Moni dict (300%) than 300 mg/m2 doxorubicin (cumulative dose) or greater than 600 mg/m2 epirubicin sdfsdf  fff  fdfsd", True)
print(a1)
print(str(a1)==str(['(300%) than', '600 mg/m2 epirubicin', '300 mg/m2 doxorubicin', '300 mg/m2', '600 mg/m2']))

test2 = containsNumericalInstances("e {5.7%} mg Age 52-75 years old; 100,000/mm³ 200 mg test 50 mg/min 2.0 mg/dl i f3.566 mg 66|8 t/8 9,7 test 8,88888888/55 9/8 3.7/9.0 11  61.1% 65.5 % [2]1/50", True)
print(test2)
print(str(test2)==str(['9,7 test', 'f3.566 mg', '3.7/9.0 11', '200 mg test', '8,88888888/55 9/8', '100,000/mm³ 200', '100,000/mm³', '2.0 mg/dl', '7/9.0 11', '50 mg/min', '52-75 years']))

test3 = containsNumericalInstances("Age 52-75 years old;  100,000/mm³ vals 200 mg 50 mg/min 2,0 mg/dl y3.222 mg i f3.566 mg/kg 66|8 mg t/8 mg 9,7 mg test 8,88888888/55 mg 9/8 mg 5.5/5.0 11 mg 3.7/9.0 11 mg 3.74/9.45666 11 mg 61.1% mg 65.5 % mg [2]1/50 mg", True)
print(test3)
print(str(test3)==str(['9/8 mg', '3.74/9.45666 11', '3.74/9.45666 11 mg', '5.5/5.0 11', '3.7/9.0 11 mg', '61.1% mg', 'y3.222 mg', '100,000/mm³ vals', 'f3.566 mg/kg', '9,7 mg', '5.5/5.0 11 mg', '100,000/mm³', '8,88888888/55 mg', '1/50 mg', '3.7/9.0 11', '65.5 % mg', '2,0 mg/dl', '74/9.45666 11', '7/9.0 11', '50 mg/min', '52-75 years', '5/5.0 11']))

test4 = containsNumericalInstances("a {5.7} mg Age 52-75 years old; 100,000/mm³ vals 200 mg (200 mg) 50 mg/min 2.0 mg/dl y3.222 mg i f3.566 mg/kg 66|8 mg t/8 mg (9,7%) [9,7%] 9,7 mg test 8,88888888/55 mg 9/8 mg 5.5/5.0 11 mg 3.7/9.0 11 mg 3.74/9.45666 11 mg 61.1% mg 65.5 % mg [2]1/50 mg", True)
print(test4)
print(str(test4)==str(['9/8 mg', '3.74/9.45666 11', '3.74/9.45666 11 mg', '5.5/5.0 11', '3.7/9.0 11 mg', '61.1% mg', 'y3.222 mg', '100,000/mm³ vals', 'f3.566 mg/kg', '9,7 mg', '[9,7%] 9,7', '5.5/5.0 11 mg', '100,000/mm³', '8,88888888/55 mg', '1/50 mg', '3.7/9.0 11', '65.5 % mg', '74/9.45666 11', '2.0 mg/dl', '7/9.0 11', '50 mg/min', '52-75 years', '5/5.0 11']))
# print(['                                  8,88888888/55 mg', '9/8 mg', '1/50 mg', '61.1% mg', '65.5 % mg', '5.5/5.0', '3.7/9.0', '3.74/9.45666', '5.5/5.0 11 mg ', '3.7/9.0 11 mg ', '3.74/9.45666 11 mg '])

['2-4 N0-3M0', '1-3M0 or', '100,000/mm¬¨‚â•, ANC', '1,700/mm¬¨‚â•', '100,000/mm¬¨‚â•,', '1,700/mm¬¨‚â• and', '4 to 5']
False
['100,000/mm³ Hemoglobin', 'Hemoglobin 9.0 g/dL', '0-1 ANC', '100,000/mm³', '2/neu positive', '12 weeks ECOG', '9.0 g/dL', '2 months of', '1,500/mm³ Platelet', '1,500/mm³']
False
['67 staining (cumulative', '5 times daily', 'ALT 2.5 times daily', 'Ki67 staining', '75 years old;', '52-75 years', '1-21. Inclusion', '1-1½ hours']
False
['2 epirubicin sdfsdf', '(300%) than', '600 mg/m2 epirubicin', '2 doxorubicin (cumulative', '300 mg/m2 doxorubicin', '300 mg/m2', '600 mg/m2']
False
['9,7 test', 'f3.566 mg', '3.7/9.0 11', 'min 2.0 mg/dl', '200 mg test', '8,88888888/55 9/8', '100,000/mm³ 200', '100,000/mm³', '2.0 mg/dl', '7/9.0 11', '50 mg/min', '8 9,7 test 8', '75 years old;', '52-75 years']
False
['mg 61.1% mg 65', '9/8 mg', '3.74/9.45666 11', '200 mg 50', '75 years old;', '3.74/9.45666 11 mg', '5.5/5.0 11', '3.7/9.0 11 mg', '61.1% mg', '8 mg 9,7', 'y3.222 mg', '100

In [163]:
ww = [' 300 mg/m2 doxorubicin', ' 600 mg/m2 epirubicin', ' 300 mg/m2', ' 600 mg/m2', '(300%) than']
wq = ww[0:]
wq.remove(' 300 mg/m2')
wq

' 300 mg/m2 doxorubicin'.strip()

'300 mg/m2 doxorubicin'

In [164]:
# def containsNInstances(sentence):
#   # s1 = re.findall(r'\d+[\.,]{0,1}\d*[\/]\d+[\.,]{0,1}\d*\s{1}\D+', sentence)
#   s1 = re.findall(r'[0-9]+[\.,]{0,1}[0-9]*[\/][0-9]+[\.,]{0,1}[0-9]*\s{1}[\s\d\^\/\-_]+[a-zA-Z]{,4}\s{1}', sentence)
#   s2 = re.findall(r'[0-9]+[\.,]{0,1}[0-9]*\s{0,1}%\s{1}[a-zA-Z]+[\^\/\-_]{0,1}[a-zA-Z]+', sentence) # or re.findall(r'[0-9]\s+', sentence) or re.findall(r'([0-9]+)\.([0-9]+)', sentence)
 
#   s3 = re.findall(r'[0-9]+[\.,]{0,1}[0-9]*[\/][0-9]+[\.,]{0,1}[0-9]+(?!\s{1}[a-zA-Z])', sentence)
#   # s4 = re.findall(r'\s[0-9]+\s{1}[a-zA-Z]+[\^\/\-_]{0,1}[a-zA-Z]+', sentence)
#   #s3 = re.findall(r'[0-9]+[\.]{0,1}[0-9]*\s{0,1}%', sentence)
#   return s1


# print(containsNInstances("i f3.566 mg/kg 66|8 t/8 9,7 test 8,88888888/55 9/8 3.7/9.0 11  61.1% 65.5 % [2]1/50"))
# print(containsNInstances("i f3.566 mg/kg 66|8 mg t/8 mg 9,7 mg test 8,88888888/55 mg 9/8 mg 5.5/5.0 11 mg 3.7/9.0 11 mg 3.74/9.45666 11 mg 61.1% mg 65.5 % mg [2]1/50 mg"))

# print(['8,88888888/55 mg', '9/8 mg', '1/50 mg', '61.1% mg', '65.5 % mg'])

In [165]:
def containsNr(word):
  return re.findall(r'\d', word)

def nnChunk(chunk):
  _word = chunk
  word = None
  withSeparator = False;
  if _word.startswith(';'):
     word = _word[1:]
  # elif _word.endswith(')'):
  #   word = _word[:len(_word)-1]
  if _word.endswith(','):
    if not containsNr(_word):
      word = _word[:len(_word)-1]+" "
    else:
      word = _word
  elif _word.endswith('.'):
    if not containsNr(_word):
      word = _word[:len(_word)-1]+" "
    else:
      word = _word
  # if _word.endswith(':'):
  #   word = _word[:len(_word)-1]
  # elif _word.endswith(';'):
  #   word = _word[:len(_word)-1]
  # elif _word.startswith('„'):
  #   word = _word[1:]
  # elif _word.endswith('”'):
  #   word = _word[:len(_word)-1]
  # elif _word.startswith('{'):
  #   word = _word[1:]
  # elif _word.startswith('['):
  #   word = _word[1:]
  # elif _word.startswith("\'"):
  #   word = _word[1:]
  # elif _word.endswith('\''):
  #   word = _word[:len(_word)-1]
  # elif _word.endswith('}'):
  #   word = _word[:len(_word)-1]
  # elif _word.endswith(']'):
  #   word = _word[:len(_word)-1]
  else:
    word = _word
  return word

# punctuation = ['„', '”', '(', ')', ';', '{', '}','[',']', '/', '\\',"\"", "'", ','] 
punctuation = ['„', '”', ';', '\\',"\"", "'"] 

def normalizeChunk(chunk):
  concats=[]
  # for sign in punctuation:
  #   if(word.__contains__(sign)):
  c = chunk.split(" ")
  for w in c:
    sw = nnChunk(w)
    concats.append(sw)
  if(len(concats) == 0):
    concats.append(chunk)
  return list(filter(None, dict.fromkeys(concats)))

print(normalizeChunk("3.5 mg, casa,Stage IV."))
print(normalizeChunk("3.5 mg. casa,Stage IV"))
print(normalizeChunk("3 casa,Stage IV"))
print(normalizeChunk("3casa.Stage IV"))
normalizeChunk(normalizeChunk("3casa.Stage IV")[0])

['3.5', 'mg ', 'casa,Stage', 'IV ']
['3.5', 'mg ', 'casa,Stage', 'IV']
['3', 'casa,Stage', 'IV']
['3casa.Stage', 'IV']


['3casa.Stage']

In [166]:
def normalizeDocument(text, removeStopWords):
  NORMALIZED_CORPUS = []
  for _word in text.split():
    if(_word.__contains__('>=')):
      _word = _word.replace(">=", "greater than or equal ")
      #print(_word)
    if(_word.__contains__('=<')):
      _word = _word.replace("=<", "smaller than or equal ")
      #print(_word)
    if(_word.__contains__('<')):
      _word = _word.replace("<", "smaller than ")
      #print(_word)
    if(_word.__contains__('>')):
      _word = _word.replace(">", "greater than ")
      #print(_word)
    normalized = normalizeChunk(_word)
    for word in normalized:
      if(removeStopWords):
        if(not STOP_WORDS_RO.__contains__(word)):
          NORMALIZED_CORPUS.append(word)
      else:
        NORMALIZED_CORPUS.append(word)
  return " ".join(NORMALIZED_CORPUS)

normalizat = normalizeDocument("Renal function: Creatinine =< 2.0 mg/dl or a creatinine clearance >= 50 mg/min", True)
normalizat

'Renal function: Creatinine smaller than or equal 2.0 mg/dl or a creatinine clearance greater than or equal 50 mg/min'

In [167]:
containsNumericalInstances(normalizat)

['50 mg/min', 'equal 2.0 mg/dl']

In [168]:
import uuid
import json
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf
import gc
import numpy as np
import pandas as pd


from transformers import AutoTokenizer, AutoModel

c_counter = 0
from datetime import datetime
current_dateTime = datetime.now()

In [169]:
def stamp_time():
    current_dateTime = datetime.now()
    return "".join("".join("".join(str(current_dateTime).split(".")[0].split(":")).split("-")).split(" "))


In [170]:
OVF=0
OVF_V=[]

In [171]:
VECTOR_MAX=200

EPOCHS = 2
LAYER_1_NODES=256
LAYER_2_NODES=2
LAYER_1_ACTIVATION='relu'
LAYER_2_ACTIVATION='softmax'
LOSS_FN='sparse_categorical_crossentropy'
#LOSS_FN='binary_crossentropy'
METRIC='accuracy'
FIT_BATCH_SIZE=16

In [172]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM
model_name = 'giacomomiolo/biobert_reupload'
# model_name = 'dmis-lab/biobert-base-cased-v1.1'
# model_name = 'bert-base-multilingual-cased'
# model_name = "allenai/scibert_scivocab_uncased"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

## Data collection utils


In [173]:
def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)


def collectEvidenceData(evidenceSection, evidenceIndexes, clinicalTrialId, evidenceSectionId, DEBUG2):
    evidences = []
    for evIndex in evidenceIndexes:
        try:
            i = int(evIndex)
            if DEBUG2:
                print(evidenceSection[i])
            norm_evid = normalizeDocument(evidenceSection[i].strip(), True)
            evidences.append(norm_evid)
        except:
            print(evidenceIndexes)
            print('\n!!! Failed to collect evidence on index, trialId, section: ' + str(i) +' '+ clinicalTrialId, evidenceSectionId)
    return evidences


def parseEvidenceSingle(hypotesis, clinicalTrialId, evidenceSectionId, primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2):
    evidences = []
    if DEBUG1:
        print("Parse evidence Single: " + clinicalTrialId + " " + evidenceSectionId)
    CT_PATH = PREFIX+'CT_data/'+clinicalTrialId+'.json'
    try:
        with open(CT_PATH, encoding='utf-8') as json_file:
            dataCT = json.load(json_file)
            if DEBUG2:
                print(primaryEvidenceIndexes)
        evidences = collectEvidenceData(dataCT[evidenceSectionId], primaryEvidenceIndexes, clinicalTrialId, evidenceSectionId, False)
        if DEBUG2:
            print(evidences)
        tok = encode_sentence("[CLS] "+hypotesis+" [SEP] " + " ".join(evidences))
        # print("[CLS] "+hypotesis+" [SEP] " + " ".join(evidences))
        # print("       aa          ")
        # if (len(tok)>300):
        #     # OVF=OVF+1
        #     OVF_V.append(len(tok))
        #     print("!!!!!!!!!!!!!!!!")
        #     print(hypotesis)
        #     print(clinicalTrialId)

    except Exception as err:
        print('\n!!! Error for file: ' + clinicalTrialId)
        print(err)
    if DEBUG2:
        print("\n")
    return evidences

In [174]:
def splitEvidence(acc, tmp, hypotesis, evidences, evIndex, DEBUG):
  if (len(evidences)==0):
    return acc
  if (evIndex>len(evidences)-1):
    acc.append(tmp)
    # print("----------1-----------EXIT split evidenta ---------------------")
    return acc;
  # print(evidences)
  # print(evIndex)
  hypTmpl = "[CLS] "+hypotesis+" [SEP] "
  tok = encode_sentence(hypTmpl + tmp + evidences[evIndex]+" ")
  if(len(tok)<VECTOR_MAX):
    tmp = tmp + evidences[evIndex]+" "
    splitEvidence(acc, tmp, hypotesis, evidences, evIndex+1, DEBUG)
  else:
    acc.append(tmp)
    tmp=""
    tok = encode_sentence(hypTmpl + tmp + evidences[evIndex]+" ")
    if(len(tok)<VECTOR_MAX):
      if (evIndex<len(evidences)):
        tmp = tmp + evidences[evIndex]
        splitEvidence(acc, tmp, hypotesis, evidences, evIndex+1, DEBUG)
      else:
        # print("----------2-----------EXIT split evidenta ---------------------")
        acc.append(evidences[evIndex])
        return acc
    else:
      if (DEBUG):
        print("!!!!!!!! hypotesis + evidence > " + str(VECTOR_MAX) + "!!!!!!!!!!")
      return acc
  return acc


In [175]:
DS_PATH = PREFIX+ '/NLI4CT_numerical_instances_dataset.json'
DS_PATH_PAIRS = PREFIX+ '/pairs_NLI4CT_numerical_instances_dataset.json'
dsNum = []
dsNumPair = [] # contains numerical instances in statement and premises

In [176]:
def numInPair(numStm, numPrem):
  if len(numStm) > 0 and len(numPrem)>0:
    return True
  return False


STM = pd.read_csv(TRAIN_DATA) #, nrows=40
print(STM.head(5))
print(STM.shape)

countPairSingle = 0
countPairComparison = 0


countPairEntailment = 0
countPairContradiction = 0

# exit()
_EVIDENCES=[]
_HYPOTESIS=[]
Y_LABELS=np.array([], dtype="int32")
CT_index = -1
for id in STM['Id']:
    evidences = []
    CT_index = CT_index+1
    CTid = STM['Primary_id'][CT_index]
    ctIndxLabel = STM['Label'][CT_index]
    hypotesis = normalizeDocument(STM['Statement'][CT_index], True)
    hypotesisLen = len(hypotesis.split(' '))
    statementsNums = containsNumericalInstances(hypotesis)
    
    if(hypotesisLen > 90):
      print('%%%% Argument mai mare de 90 tokeni')
    if (STM['Type'][CT_index] == "Single"):
      primaryEvidenceIndexes = STM['Primary_evidence_index'][CT_index].split(',')
      evidences = parseEvidenceSingle(hypotesis, CTid, STM['Section_id'][CT_index], primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      ax = splitEvidence([], "", hypotesis, evidences, 0, DEBUG2)
      print("\n ....... " + ctIndxLabel + "   -   " + CTid + "   " + id)

      if (CT_index < 37):
        print(hypotesis)
        print(statementsNums)

      for evidenceFragment in ax:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          _EVIDENCES.append(evidenceFragment)
          _HYPOTESIS.append(hypotesis)

          premisesNums = containsNumericalInstances(evidenceFragment)
          dsNumObject = {}
          dsNumObject['id']=id
          dsNumObject['primaryId']=CTid
          dsNumObject['statement_text']=hypotesis
          dsNumObject['statement_nums']=statementsNums
          dsNumObject['label']=ctIndxLabel
          dsNumObject['premise_text']=evidenceFragment
          dsNumObject['premise_nums']=premisesNums
          dsNum.append(dsNumObject)

          if(numInPair(statementsNums, premisesNums)):
            dsNumPair.append(dsNumObject)
            countPairSingle = countPairSingle+1
            if(ctIndxLabel == 'Entailment'):
              countPairEntailment = countPairEntailment+1
            else:
              countPairContradiction = countPairContradiction+1

          if(ctIndxLabel == 'Entailment'):
            Y_LABELS = np.append(Y_LABELS, [1], axis=0)
          else:
            Y_LABELS = np.append(Y_LABELS, [0], axis=0)
          
          if (CT_index < 37):
            print("     - " + evidenceFragment)
            print(premisesNums)
        else:
          print('the premise length is negligible ')
        
    
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")

    if (STM['Type'][CT_index] == "Comparison"):
      CTidSecondary = STM['Secondary_id'][CT_index]
      print("\n ....... " + ctIndxLabel + "   p: " + CTid + "   s: " + CTidSecondary + "   "+ id)

      primaryEvidenceIndexes = STM['Primary_evidence_index'][CT_index].split(',')
      secondaryEvidenceIndexes = STM['Secondary_evidence_index'][CT_index].split(',')
      premisesPrimary = parseEvidenceSingle(hypotesis, CTid, STM['Section_id'][CT_index], primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      premisesSecondary = parseEvidenceSingle(hypotesis, CTidSecondary, STM['Section_id'][CT_index], secondaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      premisesIndexed = splitEvidence([], "", hypotesis, premisesPrimary + premisesSecondary, 0, DEBUG2)

      if (CT_index < 37):
        print(hypotesis)
      for evidenceFragment in premisesIndexed:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          _EVIDENCES.append(evidenceFragment)
          _HYPOTESIS.append(hypotesis)
          
          premisesNums = containsNumericalInstances(evidenceFragment)
          dsNumObject = {}
          dsNumObject['id']=id
          dsNumObject['primaryId']=CTid
          dsNumObject['secondaryId']=CTidSecondary
          dsNumObject['label']=ctIndxLabel
          dsNumObject['statement_text']=hypotesis
          dsNumObject['statement_nums']=statementsNums
          dsNumObject['premise_text']=evidenceFragment
          dsNumObject['premise_nums']=premisesNums
          dsNum.append(dsNumObject)

          if(numInPair(statementsNums, premisesNums)):
            dsNumPair.append(dsNumObject)
            countPairComparison = countPairComparison+1
            if(ctIndxLabel == 'Entailment'):
              countPairEntailment = countPairEntailment+1
            else:
              countPairContradiction = countPairContradiction+1

          if(ctIndxLabel == 'Entailment'):
            Y_LABELS = np.append(Y_LABELS, [1], axis=0)
          else:
            Y_LABELS = np.append(Y_LABELS, [0], axis=0)
        if (CT_index < 37):
          print("     - "+ evidenceFragment)
        
      # if (CT_index == 3):
      #     exit()
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")

print('.....................................')
print('Total: '+ str(len(dsNum)))
print('pairs: '+ str(len(dsNumPair)))

try:
  with open(DS_PATH, 'w', encoding="utf-8") as _file:
    _file.write(json.dumps(dsNum, ensure_ascii=False))
  with open(DS_PATH_PAIRS, 'w', encoding="utf-8") as _file:
    _file.write(json.dumps(dsNumPair, ensure_ascii=False))
  # with open(DS_PATH) as json_file:
  #     dataCT = json.load(json_file)
except:
    print('\n!!! Error for file: ')


Y_LABELS

                                     Id        Type      Section_id  \
0  5bc844fc-e852-4270-bfaf-36ea9eface3d  Comparison    Intervention   
1  86b7cb3d-6186-4a04-9aa6-b174ab764eed      Single     Eligibility   
2  dbed5471-c2fc-45b5-b26f-430c9fa37a37  Comparison  Adverse Events   
3  20c35c89-8d23-4be3-b603-ac0ee0f3b4de      Single     Eligibility   
4  f17cb242-419d-4f5d-bfa4-41494ed5ac0e  Comparison    Intervention   

    Primary_id Secondary_id  \
0  NCT01928186  NCT00684983   
1  NCT00662129          NaN   
2  NCT00093145  NCT00703326   
3  NCT01097642          NaN   
4  NCT00852930  NCT02308020   

                                           Statement          Label  \
0  All the primary trial participants do not rece...  Contradiction   
1  Patients with Platelet count over 100,000/mm¬¨...  Contradiction   
2  Heart-related adverse events were recorded in ...     Entailment   
3  Adult Patients with histologic confirmation of...  Contradiction   
4  Laser Therapy is in each coh

array([0, 0, 0, ..., 1, 1, 0])

In [177]:
# pair = numerical instances found in statement and in premises
print('PairSingle study: '+str(countPairSingle))
print('PairComparison study: '+str(countPairComparison))
print('Pairs labeled as Entailment: '+str(countPairEntailment))
print('Pairs labeled as Contradiction: '+str(countPairContradiction))

print(countPairSingle + countPairComparison)
print(countPairEntailment + countPairContradiction)

PairSingle study: 812
PairComparison study: 409
Pairs labeled as Entailment: 562
Pairs labeled as Contradiction: 659
1221
1221


## Encoding

In [23]:
def encode_data_to_tensors(statements, evidences, tokenizer, dim):
    
    tokStm = [encode_sentence(s)
        for s in statements]

    tokPrem = [encode_sentence(s)
        for s in evidences]
    tokZip = list(zip(tokStm, tokPrem))

    # tokIntersect = [np.where(np.in1d(xt[0], np.intersect1d(xt[0], xt[1])), np.zeros_like(xt[0]), [1])
    #     for xt in tokZip]
    # type_s1 = tf.ragged.constant(tokIntersect);

    # tokIntersect = [np.where(np.in1d(xt[1], np.intersect1d(xt[0], xt[1])), np.zeros_like(xt[1]), [1])
    #     for xt in tokZip]
    # type_s2 = tf.ragged.constant(tokIntersect);

    statementsTokenizedTensor = tf.ragged.constant(tokStm)
    evidencesTokenizedTensor = tf.ragged.constant(tokPrem)


    cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*statementsTokenizedTensor.shape[0]
    _word_ids = tf.concat([cls, statementsTokenizedTensor, evidencesTokenizedTensor], axis=-1)
    ii = 0
    max = 0
    for ip in _word_ids:
        ii = ii + 1
        if (ip.shape[0] >= max):
            max = ip.shape[0]
            # print(ii)
            # print(max)
    print("Dataset MAX (dim): ")
    print(max)

    # Padding - adaug 0 sau 1 la tensori pt a avea aceiasi dimensiune 
    # paggings [ top bottom ] [left right] - adaug doar la final
    paddings = tf.constant([[0, 0,], [0, dim-max]])
    input_word_ids = tf.pad(_word_ids.to_tensor(), paddings, "CONSTANT", constant_values=0)
    input_mask = tf.ones_like(input_word_ids)

    type_cls = tf.zeros_like(cls)
    type_s1 = tf.zeros_like(statementsTokenizedTensor)
    type_s2 = tf.ones_like(evidencesTokenizedTensor)
    input_type_ids = tf.pad(tf.concat(
        [type_cls, type_s1, type_s2], axis=-1).to_tensor(), paddings, "CONSTANT", constant_values=0)

    inputs = {
        'input_word_ids': input_word_ids,
        'input_mask': input_mask,
        'input_type_ids': input_type_ids}

    return inputs


## Model declaration

In [24]:
def build_cnn_model():
    # bert_encoder = TFBertModel.from_pretrained(model_name)
    # bert_encoder = AutoModel.from_pretrained(model_name)
    input_word_ids = tf.keras.Input(shape=(VECTOR_MAX,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(VECTOR_MAX,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(VECTOR_MAX,), dtype=tf.int32, name="input_type_ids")
    # print(input_word_ids)

    #embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]

    # lstm = tf.keras.layers.LSTM(40, return_sequences=True, return_state=True)(embedding[:,0,:])
    # lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(input_word_ids)

    # x = tf.keras.layers.Dense(LAYER_1_NODES, kernel_initializer = 'random_uniform', activation=LAYER_1_ACTIVATION)(embedding[:,0,:])
    x = tf.keras.layers.Dense(512, activation=LAYER_1_ACTIVATION)(input_word_ids)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(2048, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(4096, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    # x = tf.keras.layers.Dense(18048, activation=LAYER_1_ACTIVATION)(x)
    # x = tf.keras.layers.LeakyReLU(0.9)(x)
    # x = tf.keras.layers.Dense(9128, activation=LAYER_1_ACTIVATION)(x)
    # x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(1024, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(64, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    output = tf.keras.layers.Dense(2, activation=LAYER_2_ACTIVATION)(x)

    learning_rate = 4e-5


    model = tf.keras.Model(inputs=[input_word_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=LOSS_FN, metrics=[METRIC])  # decay=learning_rate/(epochs*0.5)
    # decay=learning_rate/(EPOCHS*0.15)
    return model

In [25]:
def build_model():
    bert_encoder = TFBertModel.from_pretrained(model_name)
    # bert_encoder = AutoModel.from_pretrained(model_name)
    input_word_ids = tf.keras.Input(shape=(VECTOR_MAX,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(VECTOR_MAX,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(VECTOR_MAX,), dtype=tf.int32, name="input_type_ids")
    # print(input_word_ids)

    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]

    # lstm = tf.keras.layers.LSTM(256, return_sequences=True, return_state=True)(embedding[:,0,:])
    x = tf.keras.layers.Dense(256, activation=LAYER_1_ACTIVATION)(embedding[:,0,:])
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(1024, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(2048, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(256, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(128, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.7)(x)
    output = tf.keras.layers.Dense(2, activation=LAYER_2_ACTIVATION)(x)

    learning_rate = 2e-5


    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=LOSS_FN, metrics=[METRIC])  # decay=learning_rate/(epochs*0.5)

    return model


# Fit and Run model

In [26]:
train_input = encode_data_to_tensors(_HYPOTESIS, _EVIDENCES, tokenizer, VECTOR_MAX)
gc.collect()
# check shape data 
print('DEBUG train shapes: ')
print(train_input['input_word_ids'].shape)
print(train_input['input_mask'].shape)
print(train_input['input_type_ids'].shape)

Dataset MAX (dim): 
199
DEBUG train shapes: 
(2961, 200)
(2961, 200)
(2961, 200)


In [27]:
gc.collect()
# model = build_model()
model = build_cnn_model()
df = pd.DataFrame()
df['Y'] = Y_LABELS


print("dim ipoteze:")
print(len(_HYPOTESIS))
print("dim evidențe:")
print(len(_EVIDENCES))
print("dim Y:")
print(len(Y_LABELS))

model.fit(x=train_input, y=Y_LABELS, epochs = 5, verbose = 1, batch_size = FIT_BATCH_SIZE, validation_split = 0.2)

dim ipoteze:
2961
dim evidențe:
2961
dim Y:
2961
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['input_mask', 'input_type_ids'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


148/148 [==============================] - 41s 270ms/step - loss: 185.1988 - accuracy: 0.4831 - val_loss: 102.7652 - val_accuracy: 0.4772
Epoch 2/5
148/148 [==============================] - 38s 257ms/step - loss: 57.9673 - accuracy: 0.6698 - val_loss: 78.2394 - val_accuracy: 0.4890
Epoch 3/5
148/148 [==============================] - 38s 257ms/step - loss: 36.4258 - accuracy: 0.7247 - val_loss: 85.2769 - val_accuracy: 0.4503
Epoch 4/5
148/148 [==============================] - 37s 248ms/step - loss: 25.6199 - accuracy: 0.7960 - val_loss: 79.7827 - val_accuracy: 0.4789
Epoch 5/5
148/148 [==============================] - 35s 237ms/step - loss: 28.9887 - accuracy: 0.7910 - val_loss: 109.1030 - val_accuracy: 0.4772


## Evaluare, test data


In [28]:
TEST_STM = pd.read_csv(TEST_DATA) #, nrows=40
print(TEST_STM.head(5))
print(TEST_STM.shape)


# exit()
TEST_EVIDENCES=[]
TEST_HYPOTESIS=[]
TEST_UUID=[]
TEST_LABEL=[]
TEST_Y_LABELS=np.array([], dtype="int32")
CT_index = -1
for id in TEST_STM['Id']:
    evidences = []
    CT_index = CT_index+1
    CTid = TEST_STM['Primary_id'][CT_index]
    hypotesis = normalizeDocument(TEST_STM['Statement'][CT_index], True)
    hypotesisLen = len(hypotesis.split(' '))
    if (TEST_STM['Type'][CT_index] == "Single"):
      primaryEvidenceIndexes = TEST_STM['Primary_evidence_index'][CT_index].split(',')
      evidences = parseEvidenceSingle(hypotesis, CTid, TEST_STM['Section_id'][CT_index], primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      ax = splitEvidence([], "", "[CLS] "+hypotesis+" [SEP] ", evidences, 0, DEBUG2)
      for evidenceFragment in ax:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          TEST_EVIDENCES.append(evidenceFragment)
          TEST_HYPOTESIS.append(hypotesis)
          TEST_UUID.append(id)
          TEST_LABEL.append(TEST_STM['Label'][CT_index])
          if(TEST_STM['Label'][CT_index] == 'Entailment'):
            TEST_Y_LABELS = np.append(TEST_Y_LABELS, [1], axis=0)
          else:
            TEST_Y_LABELS = np.append(TEST_Y_LABELS, [0], axis=0)
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")
    if (TEST_STM['Type'][CT_index] == "Comparison"):
      CTidSecondary = TEST_STM['Secondary_id'][CT_index]
      primaryEvidenceIndexes = TEST_STM['Primary_evidence_index'][CT_index].split(',')
      secondaryEvidenceIndexes = TEST_STM['Secondary_evidence_index'][CT_index].split(',')
      premisesPrimary = parseEvidenceSingle(hypotesis, CTid, TEST_STM['Section_id'][CT_index], primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      premisesSecondary = parseEvidenceSingle(hypotesis, CTidSecondary, TEST_STM['Section_id'][CT_index], secondaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      premisesIndexed = splitEvidence([], "", hypotesis, premisesPrimary + premisesSecondary, 0, DEBUG2)
      if (CT_index < 17):
        print(hypotesis)
      for evidenceFragment in premisesIndexed:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          TEST_EVIDENCES.append(evidenceFragment)
          TEST_HYPOTESIS.append(hypotesis)
          TEST_UUID.append(id)
          TEST_LABEL.append(TEST_STM['Label'][CT_index])
          if(TEST_STM['Label'][CT_index] == 'Entailment'):
            TEST_Y_LABELS = np.append(TEST_Y_LABELS, [1], axis=0)
          else:
            TEST_Y_LABELS = np.append(TEST_Y_LABELS, [0], axis=0)
        if (CT_index < 17):
          print("     - "+ evidenceFragment)
      # print("-------2--------")
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")


print("dim ipoteze:")
print(len(TEST_EVIDENCES))
print("dim evidențe:")
print(len(TEST_EVIDENCES))
print("dim Y:")
print(len(TEST_Y_LABELS))

                                     Id        Type      Section_id  \
0  1adc970c-d433-44d0-aa09-d3834986f7a2      Single         Results   
1  6b9162d0-0816-46d4-81af-c60028dcc63b  Comparison     Eligibility   
2  0b6cc8e3-69ee-4a91-b93d-2ad3fddce65f  Comparison  Adverse Events   
3  cc1f712a-2116-4e40-9810-f315e3fa5ff8      Single         Results   
4  904061c0-14fa-4f13-9118-9a41e24fa8eb      Single     Eligibility   

    Primary_id Secondary_id  \
0  NCT00066573          NaN   
1  NCT00425854  NCT01224678   
2  NCT02273973  NCT00281697   
3  NCT00593346          NaN   
4  NCT02340221          NaN   

                                           Statement          Label  \
0  there is a 13.2% difference between the result...  Contradiction   
1  Patients with significantly elevated ejection ...  Contradiction   
2  a significant number of the participants in th...  Contradiction   
3  the primary trial does not report the PFS or o...     Entailment   
4  Prior treatment with fulvest

In [29]:
test_input = encode_data_to_tensors(TEST_HYPOTESIS, TEST_EVIDENCES, tokenizer, VECTOR_MAX)

# check shape data 
print('DEBUG test shapes: ')
print(test_input['input_word_ids'].shape)
print(test_input['input_mask'].shape)
print(test_input['input_type_ids'].shape)

Dataset MAX (dim): 
199
DEBUG test shapes: 
(308, 200)
(308, 200)
(308, 200)


In [30]:

c_counter = c_counter+1
gc.collect()
predictions = [np.argmax(i) for i in model.predict(test_input)]


submission = pd.DataFrame()
# print([i for i in model.predict(test_input)])

submission['UUID'] = TEST_UUID
submission['label'] = TEST_LABEL
submission['prediction'] = predictions

RESULTS_FILE = PREFIXG+"/submission_" +str(c_counter)+"_"+stamp_time()+".csv"
RESULTS_FILE_R = PREFIXG+"/results_" +str(c_counter)+"_"+stamp_time()+".json"
submission.to_csv(RESULTS_FILE, index = False)
submission.head()

 3/10 [========>.....................] - ETA: 0s

/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['input_mask', 'input_type_ids'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


10/10 [==============================] - 0s 35ms/step


,UUID,label,prediction
0,1adc970c-d433-44d0-aa09-d3834986f7a2,Contradiction,0
1,1adc970c-d433-44d0-aa09-d3834986f7a2,Contradiction,0
2,6b9162d0-0816-46d4-81af-c60028dcc63b,Contradiction,0
3,6b9162d0-0816-46d4-81af-c60028dcc63b,Contradiction,0
4,6b9162d0-0816-46d4-81af-c60028dcc63b,Contradiction,0


# Postprocessing

Conversion from json to csv of training, test and dev files. 

In [31]:
import json
import pandas as pd


"""
type = train | dev
"""
def preprocessing(JSONPATH, type, PREFIX):

    with open(JSONPATH) as json_file:
        data = json.load(json_file)

    columns = ['Type', 'Section_id', 'Primary_id', 'Secondary_id', 'Statement', 'Label', ] #  'Primary_evidence_index', 'Secondary_evidence_index',

    ids = []
    types = []
    sectionId = []
    primaryId = []
    secondaryId = []
    statement = []
    label = []
    PEI = []
    SEI = []
    for entry in data:
        ids.append(entry)
        types.append(data[entry]['Type'])
        sectionId.append(data[entry]['Section_id'])
        primaryId.append(data[entry]['Primary_id'])
        if not (data[entry].get('Secondary_id') is None):
            secondaryId.append(data[entry]['Secondary_id'])
        else:
            secondaryId.append(None)
        statement.append(data[entry]['Statement'])
        label.append(data[entry]['Label'])
        PEI.append(','.join(str(x) for x in data[entry]['Primary_evidence_index']))
        if not (data[entry].get('Secondary_evidence_index') is None):
            SEI.append(','.join(str(x) for x in data[entry]['Secondary_evidence_index']))
        else:
            SEI.append(','.join(str(x) for x in []))

    df = pd.DataFrame()
    df['Id'] = ids
    df['Type'] = types
    df['Section_id'] = sectionId
    df['Primary_id'] = primaryId
    df['Secondary_id'] = secondaryId
    df['Statement'] = statement
    df['Label'] = label
    df['Primary_evidence_index'] = PEI
    df['Secondary_evidence_index'] = SEI

    #print(df.head(20))

    compression_opts = dict(method='zip', archive_name='out.csv') 
    # df.to_csv('out.zip', index=False, compression=compression_opts) 
    df.to_csv(PREFIX+type+'.csv', index=False) 



def collectEvidenceData(evidenceSection, evidenceIndexes, clinicalTrialId, evidenceSectionId, DEBUG2):
    evidences = []
    for evIndex in evidenceIndexes:
        try:
            i = int(evIndex)
            if DEBUG2:
                print(evidenceSection[i])
            evidences.append(evidenceSection[i].strip())
        except:
            print(evidenceIndexes)
            print('\n!!! Failed to collect evidence on index, trialId, section: ' + str(i) + clinicalTrialId, evidenceSectionId)
    return evidences

##
# TODO redefined but not used
##
def parseEvidenceSingleValidation(clinicalTrialId, evidenceSectionId, primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2):
    if DEBUG1:
        print("Parse evidence Single: " + clinicalTrialId + " " + evidenceSectionId)
    CT_PATH = PREFIX+'/CT_data/'+clinicalTrialId+'.json'
    try:
        with open(CT_PATH) as json_file:
            dataCT = json.load(json_file)
            if DEBUG2:
                print(primaryEvidenceIndexes)
        evidences = collectEvidenceData(dataCT[evidenceSectionId], primaryEvidenceIndexes, clinicalTrialId, evidenceSectionId)
        if DEBUG2:
            print(evidences)
                
    except:
        print('\n!!! Error for file: ' + clinicalTrialId)
    if DEBUG2:
        print("\n")
    return evidences


# preprocessing("/content/dev.json", "dev", "")

## Metrics (F1 score) script

In [32]:
!pip install sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
import numpy
from sklearn.metrics import f1_score,precision_score,recall_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2950 sha256=5d72cc9d554c58644b1549a360385942b01b9a116b7849702ac57cd8a68265d0
  Stored in directory: /root/.cache/pip/wheels/38/1f/8d/4f812c590e074c1e928f5cec67bf5053b71f38e2648739403a
Successfully built sklearn


In [33]:
import json
import pandas as pd
from functools import reduce 


dev_path = PREFIXG+"dev.json"
with open(dev_path) as json_file:
    dev = json.load(json_file)


def postProcessing(file):
    results2 = {}
    results = {}
    CT_index = 0
    try:
        STM = pd.read_csv(file)
        # print(STM.head(5))
        for id in STM['UUID']:
            prediction = STM['prediction'][CT_index]
            # print(prediction)
            CT_index = CT_index + 1
            if id in results:
              results[id] = results[id] + "," +str(prediction)
            else:
              results[id] = str(prediction)
    except Exception as err:
        print('\n! Error for file')
        print(err)

    for o in results:
        # print(len(results[o]))
        value = {}
        if(len(results[o])>1):
          nums = results[o].split(',')
          preds = list(map(lambda n: int(n), nums))
          # print(preds)
          lst_len= len(preds)
          lst_avg = reduce(lambda x, y: x + y, preds)/lst_len
          if (lst_avg >= 0.5):
            value['Prediction'] = 'Entailment'
          else:
            value['Prediction'] = 'Contradiction'
        else:
          value['Prediction'] = 'Entailment' if int(results[o]) == 1 else 'Contradiction'
        results2[o] = value
    return results2


w = postProcessing(RESULTS_FILE)

def run_score_script(Results):
    gold = dev
    results = Results  
    uuid_list = list(results.keys())

    results_pred = []
    gold_labels = []
    for i in range(len(uuid_list)):
        if results[uuid_list[i]]["Prediction"] == "Entailment":
            results_pred.append(1)
        else:
            results_pred.append(0)
        if gold[uuid_list[i]]["Label"] == "Entailment":
            gold_labels.append(1)
        else:
            gold_labels.append(0)

    f_score = f1_score(gold_labels,results_pred)
    p_score = precision_score(gold_labels,results_pred)
    r_score = recall_score(gold_labels,results_pred)
    SCORE = "-".join(str(f_score).split("."))
    SCORE = SCORE[:6]

    print('F1:{:f}'.format(f_score))
    print('precision_score:{:f}'.format(p_score))
    print('recall_score:{:f}'.format(r_score))
    return SCORE


Run score script and write results in correct format

In [34]:
SCORE = run_score_script(w)

with open(RESULTS_FILE_R, 'w') as convert_file:
  convert_file.write(json.dumps(w))

F1:0.240602
precision_score:0.484848
recall_score:0.160000


# Test.json Task 1

In [35]:
def parseEvSg(clinicalTrialId, evidenceSectionId, PREFIX, DEBUG1, DEBUG2):
    evidences = []
    if DEBUG1:
        print("Load premises: " + clinicalTrialId + " " + evidenceSectionId)
    CT_PATH = PREFIX+'CT_data/'+clinicalTrialId+'.json'
    try:
        with open(CT_PATH) as json_file:
            dataCT = json.load(json_file)
        evidences = dataCT[evidenceSectionId]
        # if (len(premises) > 8):
        #   num = 0
        #   for prem in premises:
        #     if (num % 2) == 0:
        #       evidences.append(prem)
        #       num = num + 1
        #       print("PREMISE SELECTIE ----------------------------------")
        # else:
        #   print("PREMISE PUTINE ----------------------------------")
        #   evidences = premises
    except Exception as err:
        print('\n!!! Error for file: ' + clinicalTrialId)
        print(err)
    if DEBUG2:
        print("\n")
    return evidences

In [36]:
TEST_STM_VALID = pd.read_csv(TEST_DATA2) #, nrows=40


# exit()
TEST_EVIDENCES_VALID=[]
TEST_HYPOTESIS_VALID=[]
TEST_UUID_VALID=[]
CT_index = -1
for id in TEST_STM_VALID['Id']:
    evidences = []
    CT_index = CT_index+1
    CTid = TEST_STM_VALID['Primary_id'][CT_index]
    hypotesis = normalizeDocument(TEST_STM_VALID['Statement'][CT_index], True)
    hypotesisLen = len(hypotesis.split(' '))
    if (TEST_STM_VALID['Type'][CT_index] == "Single"):
      evidences = parseEvSg(CTid, TEST_STM_VALID['Section_id'][CT_index], PREFIX, DEBUG1, DEBUG2)
      ax = splitEvidence([], "", "[CLS] "+hypotesis+" [SEP] ", evidences, 0, DEBUG2)
      for evidenceFragment in ax:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          TEST_EVIDENCES_VALID.append(evidenceFragment)
          TEST_HYPOTESIS_VALID.append(hypotesis)
          TEST_UUID_VALID.append(id)
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")
    if (TEST_STM_VALID['Type'][CT_index] == "Comparison"):
      CTidSecondary = TEST_STM_VALID['Secondary_id'][CT_index]
      premisesPrimary = parseEvSg(CTid, TEST_STM_VALID['Section_id'][CT_index], PREFIX, DEBUG1, DEBUG2)
      premisesSecondary = parseEvSg(CTidSecondary, TEST_STM_VALID['Section_id'][CT_index], PREFIX, DEBUG1, DEBUG2)
      premisesIndexed = splitEvidence([], "", hypotesis, premisesPrimary + premisesSecondary, 0, DEBUG2)
      if (CT_index < 17):
        print(hypotesis)
      for evidenceFragment in premisesIndexed:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          TEST_EVIDENCES_VALID.append(evidenceFragment)
          TEST_HYPOTESIS_VALID.append(hypotesis)
          TEST_UUID_VALID.append(id)
        if (CT_index < 17):
          print("     - "+ evidenceFragment)
      # print("-------2--------")
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")


Women suffering from both claustrophobia and IBS or not eligible for either the primary trial or the secondary trial 
     - Inclusion Criteria:   Women are eligible to participate if they are 40 years or older and have been menopausal (had no menstrual bleeding during the past 12 months) Exclusion Criteria:   History of bilateral mastectomy, osteoporosis or renal impairment. Inclusion Criteria:   Women with a clinically or mammographically identified suspicious breast mass that is likely to be biopsied or surgically removed. Exclusion Criteria:   Pregnancy   Ferromagnetic implants   History of shotgun wounds and shrapnel   Obesity (>250 pounds)   Cardiac pacemaker   Incompatible implanted medical device   Severe claustrophobia   Major surgeries with potential of ferromagnetic implants   Severe asthma and allergies 
     -   i-STAT system, a handheld blood analyzer (I-STAT) creatinine test, estimated glomerular filtration rate (GFR) <30  Metallic object (greater than 2 cm in length) in

In [37]:
validation_input = encode_data_to_tensors(TEST_HYPOTESIS_VALID, TEST_EVIDENCES_VALID, tokenizer, VECTOR_MAX)

# check shape data 
print('DEBUG test shapes: ')
print(validation_input['input_word_ids'].shape)
print(validation_input['input_mask'].shape)
print(validation_input['input_type_ids'].shape)

Dataset MAX (dim): 
199
DEBUG test shapes: 
(1493, 200)
(1493, 200)
(1493, 200)


In [38]:
predictions = [np.argmax(i) for i in model.predict(validation_input)]
# print([i for i in model.predict(test_input)])

submission = pd.DataFrame()
submission['UUID'] = TEST_UUID_VALID
submission['prediction'] = predictions

RESULTS_TEST = PREFIXG+"/test-submission_" +str(c_counter)+"_"+SCORE  +".csv"
RESULTS_TEST_R = PREFIXG+"/test-rez-"+SCORE+"__"+stamp_time()+".json"
submission.to_csv(RESULTS_TEST, index = False)
submission.head()

47/47 [==============================] - 4s 84ms/step


,UUID,prediction
0,9f978634-637c-472f-a588-6f4bb2fb121f,0
1,9f978634-637c-472f-a588-6f4bb2fb121f,0
2,20b34e62-97c2-4ca0-bb1d-7824dab0b8bb,1
3,893a5337-2aa9-4a87-a020-4c2f03cd4aea,0
4,d401affc-f081-4eee-bd61-d109cc88f6de,0


In [39]:
submission.to_csv(RESULTS_TEST, index = False)
w = postProcessing(RESULTS_TEST)

with open(RESULTS_TEST_R, 'w') as convert_file:
  convert_file.write(json.dumps(w))